# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import sys
import time
import pickle
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from xgboost import XGBRFClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\E079051\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\E079051\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\E079051\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///../data/MessagesDatabase.db')
df = pd.read_sql_table('MessagesDatabase', con=engine)

In [6]:
# check that the data read in correctly
df

message   genre  related  \
0      Weather update - a cold front from Cuba that c...  direct        1   
1                Is the Hurricane over or is it not over  direct        1   
2                        Looking for someone but no name  direct        1   
3      UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4      says: west side of Haiti, rest of the country ...  direct        1   
...                                                  ...     ...      ...   
26175  The training demonstrated how to enhance micro...    news        0   
26176  A suitable candidate has been selected and OCH...    news        0   
26177  Proshika, operating in Cox's Bazar municipalit...    news        1   
26178  Some 2,000 women protesting against the conduc...    news        1   
26179  A radical shift in thinking came about as a re...    news        1   

       request  offer  aid_related  medical_help  medical_products  \
0            0      0            0             0                 0   
1            0      0            1             0                 0   
2            0      0            0             0                 0   
3            1      0            1             0                 1   
4            0      0            0             0                 0   
...        ...    ...          ...           ...               ...   
26175        0      0            0             0                 0   
26176        0      0            0             0                 0   
26177        0      0            0             0                 0   
26178        0      0            1             0                 0   
26179        0      0            0             0                 0   

       search_and_rescue  security  ...  aid_centers  other_infrastructure  \
0                      0         0  ...            0                     0   
1                      0         0  ...            0                     0   
2                      0         0  ...            0                     0   
3                      0         0  ...            0                     0   
4                      0         0  ...            0                     0   
...                  ...       ...  ...          ...                   ...   
26175                  0         0  ...            0                     0   
26176                  0         0  ...            0                     0   
26177                  0         0  ...            0                     0   
26178                  0         0  ...            0                     0   
26179                  0         0  ...            0                     0   

       weather_related  floods  storm  fire  earthquake  cold  other_weather  \
0                    0       0      0     0           0     0              0   
1                    1       0      1     0           0     0              0   
2                    0       0      0     0           0     0              0   
3                    0       0      0     0           0     0              0   
4                    0       0      0     0           0     0              0   
...                ...     ...    ...   ...         ...   ...            ...   
26175                0       0      0     0           0     0              0   
26176                0       0      0     0           0     0              0   
26177                0       0      0     0           0     0              0   
26178                0       0      0     0           0     0              0   
26179                0       0      0     0           0     0              0   

       direct_report  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  
...              ...  
26175              0  
26176              0  
26177              0  
26178              0  
26179              0  

[26180 rows x 38 columns]

In [4]:
df.shape

(26180, 38)

In [5]:
df.iloc[:,2:].sum()

related                   20067
request                    4467
offer                       118
aid_related               10843
medical_help               2083
medical_products           1312
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1669
food                       2917
shelter                    2309
clothing                    404
money                       603
missing_people              298
refugees                    875
death                      1192
other_aid                  3442
infrastructure_related     1705
transport                  1199
buildings                  1331
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7286
floods                     2149
storm                      2441
fire    

#### We can see from the output above that our dataset does not have any messages with the label 'child_alone'. Because this label is not providing any additional information, we will drop it from our final cleaned dataset to be used in modeling.

In [8]:
# None of the samples have label `child_alone`, no additional info, thus drop
df.drop('child_alone', axis=1, inplace=True)

In [11]:
# Check that 'child_alone' was removed
df.shape, df.columns

((26180, 37),
 Index(['message', 'genre', 'related', 'request', 'offer', 'aid_related',
        'medical_help', 'medical_products', 'search_and_rescue', 'security',
        'military', 'water', 'food', 'shelter', 'clothing', 'money',
        'missing_people', 'refugees', 'death', 'other_aid',
        'infrastructure_related', 'transport', 'buildings', 'electricity',
        'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
        'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
        'other_weather', 'direct_report'],
       dtype='object'))

In [12]:
# View the first message related to water
df[df.water == 1].message.iloc[0]

'Please, we need tents and water. We are in Silo, Thank you!'

In [13]:
# separate the messages from the categories
X = df.message.values
Y = df.iloc[:,2:].values

In [14]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [15]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
X.shape, Y.shape

((26180,), (26180, 35))

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    '''
    Convert a string of text into a list of tokens
    
    Input
        text: str, containing the text to be tokenized
        
    Output
        clean_tokens: list, containing normalized, tokenized and lemmatized text
    '''
    
    # detect and replace urls with placeholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # case normalize and remove punctuation, except for $ sign
    text = re.sub(r'[^a-zA-Z$]', ' ', text.lower())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # remove stop words and lemmatize
    lem = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    clean_tokens = [lem.lemmatize(tok).strip() for tok in tokens if tok not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)

In [20]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

Wall time: 17.5 s


In [23]:
# View the first ten elements of the compressed sparse row (csr)
# matrix returned by TfidfVectorizer, showing coordinates and values
print(tfidf_matrix[0])

  (0, 11176)	0.24576657955347114
  (0, 19424)	0.35062973607236964
  (0, 5730)	0.2828895174740424
  (0, 6018)	0.45997200076358874
  (0, 9996)	0.3822004543717484
  (0, 4938)	0.33653829701560334
  (0, 27971)	0.41559934724979647
  (0, 28836)	0.3049570885529014


In [24]:
tfidf_matrix.shape

(26180, 29755)

#### We can see from the shape of `tfidf_matrix` that the lexicon (vocabulary) of the data set after tokenization is 29,755. In other words, the current feature space consists of 29,755 dimensions!

#### We can look at the features that we have for the first message:

In [25]:
# View the features associated with the first message
tfidf_np_matrix = tfidf_matrix.toarray()
nonzero_indices = np.flatnonzero(tfidf_np_matrix[0])
words = tfidf_vectorizer.get_feature_names()
[words[i] for i in nonzero_indices]

['cold', 'could', 'cuba', 'front', 'haiti', 'pas', 'update', 'weather']

In [26]:
# View the first message
df.iloc[0].message

'Weather update - a cold front from Cuba that could pass over Haiti'

In [36]:
from sklearn.svm import SVC

In [69]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters = {
    'tfidf__ngram_range': ((1, 1),),
    'tfidf__max_features': (5000,),
    'clf__estimator__kernel': ['rbf'],
 }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

In [38]:
%%time
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1729)

Wall time: 21.7 ms


In [39]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20944,), (5236,), (20944, 35), (5236, 35))

In [70]:
%%time
cv.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

#### Let's see if we can reduce the dimensionality of the data set using truncated svd

In [9]:
from sklearn.decomposition import TruncatedSVD

In [25]:
%%time
truncsvd = TruncatedSVD(n_components=100)
shrunk_matrix = truncsvd.fit_transform(tfidf_matrix)

Wall time: 1.72 s


In [26]:
np.linalg.norm(shrunk_matrix[0])

0.4507982890451702

In [40]:
from sklearn.preprocessing import normalize
shrunk_normed_matrix = normalize(shrunk_matrix)

In [28]:
np.linalg.norm(shrunk_normed_matrix[0])

1.0

In [42]:
shrunk_normed_matrix.shape

(26180, 100)

In [43]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(shrunk_normed_matrix, Y, test_size=.2, random_state=1729)

In [44]:
clf = MultiOutputClassifier(XGBRFClassifier(use_label_encoder=False, verbosity=0))

In [45]:
clf.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBRFClassifier(base_score=None, booster=None,
                                                colsample_bylevel=None,
                                                colsample_bytree=None,
                                                enable_categorical=False,
                                                gamma=None, gpu_id=None,
                                                importance_type=None,
                                                interaction_constraints=None,
                                                max_delta_step=None,
                                                max_depth=None,
                                                min_child_weight=None,
                                                missing=nan,
                                                monotone_constraints=None,
                                                n_estimators=100, n_jobs=None,
                                                num_parallel_tre

In [46]:
%%time
y_pred = clf.predict(X_test)

Wall time: 289 ms


In [47]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.80      0.96      0.87      4002
               request       0.75      0.44      0.56       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.71      0.52      0.60      2177
          medical_help       0.49      0.07      0.12       405
      medical_products       0.78      0.07      0.13       264
     search_and_rescue       0.00      0.00      0.00       137
              security       0.00      0.00      0.00        98
              military       0.00      0.00      0.00       185
                 water       0.72      0.42      0.53       315
                  food       0.74      0.52      0.61       596
               shelter       0.83      0.31      0.45       472
              clothing       0.90      0.12      0.21        74
                 money       0.67      0.03      0.06       127
        missing_people       0.00      

C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [192]:
clf = MultiOutputClassifier(XGBRFClassifier(use_label_encoder=False, verbosity=0))
clf.get_params

<bound method BaseEstimator.get_params of MultiOutputClassifier(estimator=XGBRFClassifier(base_score=None, booster=None,
                                                colsample_bylevel=None,
                                                colsample_bytree=None,
                                                enable_categorical=False,
                                                gamma=None, gpu_id=None,
                                                importance_type=None,
                                                interaction_constraints=None,
                                                max_delta_step=None,
                                                max_depth=None,
                                                min_child_weight=None,
                                                missing=nan,
                                                monotone_constraints=None,
                                                n_estimators=100, n_jobs=None,
                      

In [204]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(XGBRFClassifier(use_label_encoder=False, verbosity=0)))
])

parameters = {
    'tfidf__max_df': [0.75],
 }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [113]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

parameters = {
        'tfidf__ngram_range': ((1, 1), (1, 2)),
        'tfidf__max_df': (0.5, 0.75, 1.0),
        'tfidf__max_features': (None, 5000, 10000),
    }
    
cv = GridSearchCV(pipeline, param_grid=parameters)

In [191]:
clf = XGBRFClassifier()
clf.get_params()

{'colsample_bynode': 0.8,
 'learning_rate': 1.0,
 'reg_lambda': 1e-05,
 'subsample': 0.8,
 'use_label_encoder': True,
 'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bytree': None,
 'enable_categorical': False,
 'gamma': None,
 'gpu_id': None,
 'importance_type': None,
 'interaction_constraints': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'scale_pos_weight': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [187]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1729)

In [205]:
# Train pipeline
cv.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [131]:
# predict on test data
start_time = time.time()
y_pred = pipeline.predict(X_test)
time_elapsed = round(((time.time() - start_time) / 60), 2)
print(f'Time elapsed: {time_elapsed} minutes')

Time elapsed: 0.68 minutes


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
pipeline.classes_

[array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dt

In [132]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4002
               request       0.81      0.45      0.58       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.72      0.52      0.61      2177
          medical_help       0.58      0.15      0.24       405
      medical_products       0.77      0.25      0.38       264
     search_and_rescue       0.69      0.24      0.36       137
              security       0.00      0.00      0.00        98
              military       0.62      0.21      0.31       185
                 water       0.78      0.56      0.65       315
                  food       0.77      0.78      0.78       596
               shelter       0.80      0.56      0.66       472
              clothing       0.74      0.69      0.71        74
                 money       0.59      0.19      0.29       127
        missing_people       0.61      

C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\E079051\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [133]:
y_test.sum()

16597

In [134]:
y_pred.sum()

11859

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [135]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000002BDF3C564C0>)),
                ('clf',
                 MultiOutputClassifier(estimator=XGBRFClassifier(base_score=None,
                                                                 booster=None,
                                                                 colsample_bylevel=None,
                                                                 colsample_bytree=None,
                                                                 enable_categorical=False,
                                                                 gamma=None,
                                                                 gpu_id=None,
                                                                 importance_type=None,
                                                                 interaction_constraints=None,
                                                           

In [136]:
text = 'please help. we are in desperate need of water'

In [138]:
pred = pipeline.predict([text])

In [165]:
text = 'A hurrcane swept through destroying our neighborhood and we are hungry. Please help.'
pred = pipeline.predict([text])
preds = dict(zip(list(df.iloc[:,2:].columns),pred.tolist()[0]))
preds

{'related': 1,
 'request': 1,
 'offer': 0,
 'aid_related': 1,
 'medical_help': 0,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'water': 0,
 'food': 1,
 'shelter': 0,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 0,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 0,
 'floods': 0,
 'storm': 0,
 'fire': 0,
 'earthquake': 0,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 1}

In [173]:
import pickle

pickle.dump(pipeline, open('model.pkl', 'wb'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [120]:
from xgboost import XGBClassifier, XGBRFClassifier

In [101]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
# from xgboost import XGBClassifier, XGBRFClassifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing

# Predefined function
def model_assess(model, name='Default'):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print('---', name, '---', '\n',
          confusion_matrix(y_test, preds), '\n',
          'Accuracy:', round(accuracy_score(y_test, preds), 5), '\n')
    
    # Naive Bayes
nb = GaussianNB()
model_assess(nb, name='Naive Bayes')

# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, name='SGD')

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, name='KNN')

# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, 'Decission Trees')

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, 'Random Forest')

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, 'SVM')

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, 'Logistic Regression')

# Neural Nets
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(150, 10), random_state=1)
model_assess(nn, 'Neural Nets')

# Cross Gradient Booster
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assess(xgb, 'XGBoost')

# Cross Gradient Booster (Random Forest) <=================== BEST
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
model_assess(xgbrf, 'XGBoost RF')

from sklearn.model_selection import GridSearchCV

# Grid Search Function
def gridSearch(parameters):
    gsearch = GridSearchCV(estimator = XGBRFClassifier(learning_rate =0.00001,n_estimators=1000,max_depth=1,min_child_weight=4,
                                                    gamma=0.3,subsample=0.8,colsample_bytree=0.8,objective= 'multi:softmax',
                                                    nthread=4,scale_pos_weight=1), 
                        param_grid = parameters, cv=5)
    gsearch.fit(X_train,y_train)
    print('best params:', gsearch.best_params_)
    predictions = gsearch.predict(X_test)
    print('Accuracy:', round(accuracy_score(y_test, predictions), 5))

In [ ]:
param_test1 = {
 'min_child_weight':[1, 2, 3, 4]
}

gridSearch(param_test1)

param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gridSearch(param_test2)

param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gridSearch(param_test3)

param_test5 = {
 'learning_rate':[1e-5, 1e-2, 0.1, 1, 100]
}

gridSearch(param_test5)

# Final Model
xgbrf_tuned = XGBRFClassifier(objective= 'multi:softmax', learning_rate =0.00001, n_estimators=1000, max_depth=1, 
                              min_child_weight=5,gamma = 0.3, subsample=0.8, colsample_bytree=0.9, 
                              nthread=4, scale_pos_weight=1)
model_assess(xgbrf_tuned)

# How important is each variable into predicting if the respondent has a current Mental Health Problem?

import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(estimator=xgbrf_tuned, random_state=1)
perm.fit(X_test, y_test)

eli5.show_weights(estimator=perm, feature_names = X_test.columns.tolist())



In [ ]:
# Creating new train datasets only with features that have an importance bigger than 0.004
X_train_new = X_train[['mh_disorder_past', 'mh_not_eff_treat_impact_on_work', 'mh_diagnos_proffesional', 
                       'mh_eff_treat_impact_on_work']]
X_test_new = X_test[['mh_disorder_past', 'mh_not_eff_treat_impact_on_work', 'mh_diagnos_proffesional', 
                       'mh_eff_treat_impact_on_work']]

# Fitting the model to the new train datasets to see if we get a better accuracy
xgbrf_perm_imp = XGBRFClassifier(objective= 'multi:softmax', learning_rate =0.00001, n_estimators=1000, max_depth=1, 
                              min_child_weight=5,gamma = 0.3, subsample=0.8, colsample_bytree=0.9, 
                              nthread=4, scale_pos_weight=1)
xgbrf_perm_imp.fit(X_train_new, y_train)
predictions = xgbrf_perm_imp.predict(X_test_new)
print(confusion_matrix(y_test, predictions))
print('Accuracy:', accuracy_score(y_test, predictions))

In [ ]:
from xgboost import plot_tree, plot_importance

fig, (ax1, ax2) = plt.subplots(figsize = (13, 8), ncols=1, nrows=2)
plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top = 0.9, wspace=0, hspace = 0.5)

plot_importance(xgbrf_tuned, importance_type='gain', ax = ax1)
ax1.set_title('Feature Importance by Information Gain', fontsize = 18)
ax1.set_xlabel('Gain')

plot_importance(xgbrf_tuned, importance_type='weight', ax = ax2)
ax2.set_title('Feature Importance by Weight', fontsize = 18)
ax2.set_xlabel('Weight');

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.